## Import Libraries

In [1]:
from os import path, makedirs
from itertools import product
import numpy as np
from brainpipe.system import study

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.model_selection import permutation_test_score
from sklearn.metrics import roc_auc_score

In [2]:
"""
Compute classification power in time for allfreqs and electrodes
"""

st = study('Ripples')
pow_path = path.join(st.path,'feature/')
reps = ['Encoding/', 'Retrieval_new_odors/', 'Retrieval_new_rec/']
conds = ['low','high']
freqs = ['ripple', 'low_freq','HFA']
subjects = ['LEFC','CHAF','VACJ','SEMC','FERJ','PIRJ']
nperm = 100

for rep in reps:
    pow_name = path.join(pow_path+rep, '{}_cond={}_bipo_feat_norm.npz')
    path_save = path.join(st.path, 'classified/'+rep)
    if not path.exists(path_save):
        makedirs(path_save)
    clf_name = path.join(path_save, '{}_LDA_clf_{}_{}_{}.npz')    

    for su in subjects:
        mat0 = np.load(pow_name.format(su,conds[0]), allow_pickle=True)
        mat1 = np.load(pow_name.format(su,conds[1]), allow_pickle=True)
        if rep != 'Retrieval_new_rec/':
            to_take, time = [54, 99], mat0['time']-3
        else:
            to_take, time = [39, 119], mat0['time']-5

        for i,freq in enumerate(freqs):
            id_f = [i for i,f in enumerate(mat1['fname']) if f==freq][0]
            pow_list = []
            #=========================== Load Power files (nfreq, nelec, nwin, ntrial) =================================    
            pow_list.append(mat0['xpow'][id_f,:,to_take[0]:to_take[1],:])
            nelecs, npts, _ = mat0['xpow'][id_f,:,to_take[0]:to_take[1],:].shape
            pow_list.append(mat1['xpow'][id_f,:,to_take[0]:to_take[1],:])
            print (su,mat0.files, 'xpow shape: ', [pow.shape for pow in pow_list])
        
            #=========================== Create dict for all results =================================    
            name_classif = clf_name.format(su,freq,conds[0],conds[1])
            if not path.exists(name_classif):
                kwargs = {}
                kwargs['names'], kwargs['channels'] = mat0['new_labels'], mat0['channels']
                kwargs['xyz'], kwargs['time'] = mat0['xyz'], time[to_take[0]:to_take[1]]

                # =========================== Select Power for 1 elec 1 freq =================================                 
                permut,auc = np.zeros((nelecs,npts,nperm)),np.zeros((nelecs,npts))
                for elec_num in range(nelecs):
                    print('--» processing',rep,su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                    pow_data_elec = [power[elec_num].swapaxes(0,1) for power in pow_list]

                    # create a data matrix, concatenate along the trial dimension
                    x = np.concatenate(pow_data_elec, axis=0)
                    print ('Size of the concatenated data: ', x.shape)
                    y = np.hstack([np.array([i]*len(power)) for i, power in enumerate(pow_data_elec)])
                    print ('Size of label for classif: ', len(y))

                    for t in range(npts):
                        X_t = x[:,t][:,np.newaxis]
                        clf = LDA()
                        cv = SKFold(5)
                        score, permutation_scores, pvalue = permutation_test_score(
                        clf, X_t, y, scoring="roc_auc", cv=cv, n_permutations=nperm, n_jobs=-1)
                        permut[elec_num,t] += permutation_scores
                        auc[elec_num,t] += score
    #                     print("clf score for %t %s (pvalue : %s)" % (t, score, pvalue))
                kwargs['perm'], kwargs['auc'] = permut, auc
                #Save plots
                np.savez(name_classif, **kwargs)
                del x, y, pow_data_elec, permutation_scores, score
            else:
                print(name_classif, 'already computed')
        del pow_list

-> Ripples loaded
LEFC ['f', 'baseline', 'split', 'norm', 'width', 'step', 'time', 'xpow', 'pval_xpow', 'fname', 'new_labels', 'channels', 'labels', 'xyz'] xpow shape:  [(109, 45, 34), (109, 45, 21)]
--» processing Encoding/ LEFC elec 0 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 1 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 2 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 3 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 4 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 5 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 6 /

--» processing Encoding/ LEFC elec 63 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 64 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 65 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 66 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 67 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 68 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 69 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 70 / 109 freq ripple
Size of the concatenated data:  (55, 45)
Size of

--» processing Encoding/ LEFC elec 17 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 18 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 19 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 20 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 21 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 22 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 23 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 24 / 109 freq low_freq
Size of the concatenated data:  

--» processing Encoding/ LEFC elec 81 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 82 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 83 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 84 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 85 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 86 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 87 / 109 freq low_freq
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 88 / 109 freq low_freq
Size of the concatenated data:  

--» processing Encoding/ LEFC elec 35 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 36 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 37 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 38 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 39 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 40 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 41 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55
--» processing Encoding/ LEFC elec 42 / 109 freq HFA
Size of the concatenated data:  (55, 45)
Size of label for classif:  55


...

--» processing Retrieval_new_rec/ FERJ elec 49 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 50 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 51 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 52 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 53 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 54 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 55 / 62 freq low_freq
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ 

--» processing Retrieval_new_rec/ FERJ elec 47 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 48 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 49 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 50 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 51 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 52 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 53 / 62 freq HFA
Size of the concatenated data:  (24, 80)
Size of label for classif:  24
--» processing Retrieval_new_rec/ FERJ elec 54 / 62 freq HFA
Size of 

--» processing Retrieval_new_rec/ PIRJ elec 45 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 46 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 47 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 48 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 49 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 50 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 51 / 65 freq ripple
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 52 /

--» processing Retrieval_new_rec/ PIRJ elec 39 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 40 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 41 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 42 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 43 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 44 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 45 / 65 freq low_freq
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ 

--» processing Retrieval_new_rec/ PIRJ elec 34 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 35 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 36 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 37 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 38 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 39 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 40 / 65 freq HFA
Size of the concatenated data:  (28, 80)
Size of label for classif:  28
--» processing Retrieval_new_rec/ PIRJ elec 41 / 65 freq HFA
Size of 